In [14]:
import numpy as np
import pandas as pd

In [15]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/analysis.p')
df = df.query("Domain != 'Economic'")
df = df.query("alpha != ''")
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [3]:
def getcount(decision, direction, df=df):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [4]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
expfail = getcount('dexp', 'fail')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
lnfail = getcount('dln', 'fail')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
strexpfail = getcount('dstrexp', 'fail')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [expin, lnin, strexpin, plwcin]
outcounts = [expout, lnout, strexpout, plwcout]
unkcounts = [expunk, lnunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts
print [expfail, lnfail, strexpfail]

[1066, 1396, 1291, 1702]
[2336, 479, 2020, 0]
[762, 2511, 1014, 2759]
[303, 81, 142]


In [5]:
unique_datasets = np.unique(df.fp_gml)
decisions = ['dexp','dln', 'dstrexp', 'dplwc']
allcounts = np.zeros((3,4))
# each column of counts (stack in, on, and unk) adds up to len(rows) unless some fail
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    counts = np.zeros((3,4)) # rows are in, out, and unk 
    for i, decision in enumerate(decisions):
        counts[0,i] = getcount(decision, 'in', rows)
        counts[1,i] = getcount(decision, 'out', rows)
        counts[2,i] = getcount(decision, 'unk', rows)
        counts[2,i]+= getcount(decision,'fail',rows)
    counts=counts/len(rows) # normalize each dataset
    allcounts += counts

In [6]:
100*allcounts/len(unique_datasets)

array([[ 36.48695316,  45.02692054,  42.2083584 ,  51.28070142],
       [ 36.5480601 ,  12.01204834,  33.32775653,   0.        ],
       [ 26.92838339,  42.92442777,  24.42728172,  48.68269523]])

In [7]:
doms = np.unique(df.Domain)
# number of graphs and mean alpha value in each domain
counts = np.zeros_like(doms)
means = np.zeros_like(doms)
counts_plaus = np.zeros_like(doms)
means_plaus = np.zeros_like(doms)
counts_plaustail = np.zeros_like(doms)
means_plaustail = np.zeros_like(doms)
for i,dom in enumerate(doms):
    query = "Domain == '%s'" %dom
    subdf = df.query(query)
    counts[i] = len(subdf)
    means[i] = format(np.mean(subdf.alpha), '.2f')
    subdf_plaus = subdf.query("ppl>0.1")
    counts_plaus[i] = len(subdf_plaus)
    means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
    subdf_plaustail = subdf_plaus.query("ntail>=50")
    counts_plaustail[i] = len(subdf_plaustail)
    means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')

In [8]:
print doms
print counts_plaustail
print means_plaustail

['Biological' 'Informational' 'Social' 'Technological' 'Transportation']
[125 13 239 1316 1]
['2.70' '2.69' '4.19' '2.19' '4.82']


In [16]:
def test_strong(rows):
    S1 = False # strongest
    S2 = False # strong
    SA = False
    n = len(rows)
    strong = 0
    strong_alone = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
            strong_alone += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
                strong += 1
    if strong_alone >= 9.*n/10:
        SA = True
    if strong >= n/2.:
        S2 = True
    if SA == True and strong >= 95.*n/100:
        S1 = True
    return (S1, S2)
            
def test_weak(rows):
    W = False 
    West = False 
    SW = False
    n = len(rows)
    weak = 0
    weakest = 0
    sweak = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
        if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
            sweak += 1
    if weak >= n/2.:
        W = True
    if weakest >= n/2.:
        West = True
    if sweak >= n/2.:
        SW = True
    return (W, West, SW)
          

In [20]:
# test diff hypotheses
unique_datasets = np.unique(df.fp_gml)
hyps = pd.DataFrame(columns = ['Strongest', "Strong", "Weak", "Weakest", "Super_Weak", "Domain", "Subdomain", "median_alpha", "n", "median_ntail"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [S1, S2] = test_strong(rows)
    hyps.loc[dataset]['Strongest'] = S1
    hyps.loc[dataset]['Strong'] = S2
    [weak,weakest,superweak] = test_weak(rows)
    hyps.loc[dataset]['Weak'] = weak
    hyps.loc[dataset]['Weakest'] = weakest
    hyps.loc[dataset]['Super_Weak'] = superweak
    hyps.loc[dataset]['Domain'] = rows.Domain[0]
    hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]
    hyps.loc[dataset]['median_alpha'] = np.median(rows.alpha)
    hyps.loc[dataset]['n'] = np.max(rows.n)
    hyps.loc[dataset]['median_ntail'] = np.median(rows.ntail)
    

In [21]:
n = float(len(hyps))
Strongest = np.sum(hyps.Strongest)
Strong = np.sum(hyps.Strong)
Weak = np.sum(hyps.Weak)
Weakest = np.sum(hyps.Weakest)
Super_Weak = np.sum(hyps.Super_Weak)
print "Strongest = %s" %(Strongest/n)
print "Strong = %s" %(Strong/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Super Weak = %s" %(Super_Weak/n)
print "Number of Datsets = %s" %int(n)

Strongest = 0.0377562028047
Strong = 0.105717367853
Weak = 0.24163969795
Weakest = 0.333333333333
Super Weak = 0.522114347357
Number of Datsets = 927


In [12]:
hyps.to_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/hyps.p')
hyps.to_csv('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/hyps.csv')

In [13]:
len(hyps)

927

In [78]:
df.query("fp_gml == '/Volumes/Durodon/gmls/Transportation/Roads/n6/United_States_Road_Networks_PA_Transportation_Roads_n6.gml'")

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_road_category,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_road_category,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_travel_time,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_travel_time,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_spatial_distance_in_meters,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_spatial_distance_in_meters,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1


In [12]:
subhyps = hyps.query("Domain=='Social'")
n = float(len(subhyps))
Strong1 = np.sum(subhyps.Strong1)
Strong2 = np.sum(subhyps.Strong2)
Strong_alone = np.sum(subhyps.Strong_alone)
Weak = np.sum(subhyps.Weak)
Weakest = np.sum(subhyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %int(n)

Strong1 = 0.0
Strong2 = 0.0
Strong_alone = 0.0
Weak = 0.551724137931
Weakest = 0.696551724138
Number of Datsets = 145
